In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2
import sys
# Añadimos la carpeta que contiene nuestro .py al path de Python
sys.path.append("../src/")
import support_scrap as ss
import support_mongo as sm
import support_filter_models as sfm
import support_tag as st

Some weights of ViTForImageClassification were not initialized from the model checkpoint at facebook/dino-vitb16 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [ ]:
# Importar datos bbdd
bd=sm.conectar_a_mongo("PullnBearData")
nombre_coleccion1="modelos_pull_hombre_tageados"
nombre_coleccion2="productos_pull_hombre_tageados"
df_modelos_base = sm.importar_a_dataframe(bd, nombre_coleccion1)
df_base = sm.importar_a_dataframe(bd, nombre_coleccion2)

# Mostrar las primeras filas del DataFrame
print(df_base.shape)
df_base.head()

In [ ]:
# URLs a procesar
listaurl = [
    "https://www.pullandbear.com/es/hombre/ropa/camisetas-n6323",
    "https://www.pullandbear.com/es/hombre/rebajas/ropa/camisetas-y-polos-n7087",
    "https://www.pullandbear.com/es/hombre/ropa/camisas-n6313",
    "https://www.pullandbear.com/es/hombre/rebajas/ropa/camisas-n7088",
    "https://www.pullandbear.com/es/hombre/ropa/punto-n6372",
    "https://www.pullandbear.com/es/hombre/rebajas/ropa/punto-n7090",
    "https://www.pullandbear.com/es/hombre/ropa/sudaderas-n6382",
    "https://www.pullandbear.com/es/hombre/rebajas/ropa/sudaderas-n7089",
    "https://www.pullandbear.com/es/hombre/ropa/pantalones-n6363",
    "https://www.pullandbear.com/es/hombre/rebajas/ropa/pantalones-n7091",
    "https://www.pullandbear.com/es/hombre/ropa/jeans-n6347",
    "https://www.pullandbear.com/es/hombre/rebajas/ropa/jeans-n7818"
]

df_nuevo = ss.process_and_extract_data_main(listaurl)

# Mostrar articulos antes y despues
print("Articulos base:")
print(df_base.shape)
print("Articulos actuales:")
print(df_nuevo.shape)

# Mostrar el DataFrame
df_nuevo.head()


### COMPROBACIÓN DE ESTADO CON LA BASE

In [ ]:
# Añadir la columna de stock al df_base
# Si la URL está en df_nuevo, significa que está en stock; de lo contrario, no lo está
df_base['Stock Status'] = df_base['url'].apply(
    lambda x: 'In Stock' if x in df_nuevo['url'].values else 'Out of Stock')

# Actualizar los precios en df_base si el precio en df_nuevo es diferente
for index, row in df_base.iterrows():
    if row['url'] in df_nuevo['url'].values:
        # Tomar el precio de df_nuevo (current_price)
        nuevo_precio = df_nuevo.loc[df_nuevo['url'] == row['url'], 'product_price'].values[0]
        
        # Comparar con el precio actual en df_base (product_price)
        if row['product_price'] != nuevo_precio:
            df_base.at[index, 'product_price'] = nuevo_precio

# Nuevas URLs en df_nuevo que no están en df_base
new_urls = df_nuevo[~df_nuevo['url'].isin(df_base['url'])]

# URLs en df_base que no están en df_nuevo (para marcar como "out of stock")
out_of_stock_urls = df_base[~df_base['url'].isin(df_nuevo['url'])]

# Guardar los resultados en CSV
new_urls.to_csv('../results/new_urls.csv', index=False)
out_of_stock_urls.to_csv('../results/out_of_stock_urls.csv', index=False)

# Mostrar los resultados
print("Nuevas URLs en df_nuevo que no están en df_base:")
print(len(new_urls))
print("\nURLs en df_base que no están en df_nuevo (out of stock):")
print(len(out_of_stock_urls))

### Scrapeo de las nuevas urls

In [ ]:
# Ejecutamos la funcion de scrapeo de productos
df_results=ss.process_urls_products(new_urls)

# Unir los DataFrames utilizando la columna 'url'
df_results=pd.merge(df_results,new_urls,on='url', how='inner')
df_successful = pd.concat([df_base, df_results], axis=0, ignore_index=True)

# Columnas clave para identificar duplicados
columnas_clave = ['product_price','description','current_price','color','image_url','mpn','reference_code','category_id']

# Crear un DataFrame con los duplicados eliminados
df_sin_duplicados = df_successful.drop_duplicates(subset=columnas_clave, keep='first')

# Identificar las filas descartadas como la diferencia entre los originales y los sin duplicados
descartes = df_successful.loc[~df_successful.index.isin(df_sin_duplicados.index)]

# Guardar los descartes en un archivo CSV
descartes.to_csv('../results/articulos_descartados.csv', index=False)

# Mostrar resultados
print("Articulos de tras actualización:")
print(df_successful.shape)
print(f"Se guardaron {len(descartes)} filas descartadas en 'descartes.csv'.")
print("Articulos sin duplicados:")
print(df_sin_duplicados.shape)

# HOMOGENEIZAR LOS DATOS

### COLOR

In [ ]:
# Crear una nueva columna con el color homogeneizado
df_sin_duplicados['color_homogeneizado'] = df_sin_duplicados['color'].apply(ss.homogeneizar_color)

### CATEGORÍA

In [ ]:
# Crear una nueva columna con la categoría de la prenda
df_sin_duplicados['Categoria'] = df_sin_duplicados['product_name'].apply(ss.categorizar_ropa)

# Mostrar las primeras filas para comprobar
df_sin_duplicados[['product_name', 'Categoria']].sample(5)


### CONTROLAR URLs FALTANTES

In [10]:
# Asignar "out of stock" a la columna 'Stock Status' donde 'image_url' sea vacío o nulo
df_sin_duplicados.loc[(df_sin_duplicados['image_url'].isna()) | (df_sin_duplicados['image_url'] == ''), 'Stock Status'] = 'Out of Stock'

### CREAR DF DE MODELOS NUEVO

In [ ]:
# Filtrar los pantalones para coger los modelos de cuerpo entero
df_modelos_nuevo = df_sin_duplicados[(df_sin_duplicados['Categoria'] == 'Pantalón') & (df_sin_duplicados['Stock Status'] == 'In Stock')] 
df_modelos_nuevo["Fondo"] =df_modelos_nuevo["image1_url"].apply(sfm.is_white_background)
df_modelos_nuevo["Modelo"] =df_modelos_nuevo["image1_url"].apply(sfm.classify_image)
df_modelos_final=df_modelos_nuevo[(df_modelos_nuevo["Fondo"] == "Blanco")&(df_modelos_nuevo["Modelo"] == "Modelo")&(df_modelos_nuevo["Stock Status"] == "In stock")]

### GUARDAR CSV

In [ ]:
# Guardar el nuevo DataFrame con pantalones en un archivo CSV
df_modelos_final.to_csv('../results/Modelos_enteros.csv', index=False)
# Guardar los cambios en un archivo nuevo si lo necesitas
df_sin_duplicados.to_csv('../results/all_products_info_with_categories.csv', index=False)

In [ ]:
# Guardamos los datos en la bbdd de Mongo
bd=sm.conectar_a_mongo("PullnBearData")
nombre_coleccion1="modelos_pull_hombre_tageados"
nombre_coleccion2="productos_pull_hombre_tageados"
sm.eliminar_coleccion(bd, nombre_coleccion1)
sm.subir_dataframe_a_mongo(bd, df_modelos_final, nombre_coleccion1)
sm.eliminar_coleccion(bd, nombre_coleccion2)
sm.subir_dataframe_a_mongo(bd, df_sin_duplicados, nombre_coleccion2)